# Obtaining the lowest depth

In [1]:
import pandas as pd
from qiime2 import Artifact

In [2]:
class Data:
    def __init__(self):
        self.qzv_in = "Assets/feature_table_full.qza"
        self.mode = "full"
args = Data()

In [5]:
# Open feature table
feature_table_df = Artifact.load(args.qzv_in).view(pd.DataFrame)
feature_table_df["Total depth"] = feature_table_df.sum(axis=1).astype(int)
depths_df = pd.DataFrame(feature_table_df.loc[:,"Total depth"])
depths_df.sort_values("Total depth").transpose().to_csv(f"depths_full.tsv", sep="\t")

In [4]:
feature_table_df

,d5c34878762407bfbdc394e5b9038ee2,a7cf2890d69762294b1e0e201aad95e0,331553b3c1ef1e68069b252e3121608d,e37e4533b30c32a38231d41857c1b756,160498de343e5be8c3b571d9707af397,fc83d8a39b182ce5ca65b71dfc57556f,f490b86b593309cd63b859eb3e14b073,b66f72f3cf953f219908b4949c30f5fe,5aacd6ad1c9844395b3b95d335dc843f,5bc339323ebb759f9469a29c2eb2c4de,...,373139cd2d0c1bc3651baeb32a4bf958,d5a922f04ad47c027b151b425bbaf1e5,12d3d41757fd93eb0ed75eed07b5e58b,a3395f8e9e0b527e5f0c959a3ade6e14,f4422e09367fbe080b0ffafb187563bc,a219f4539c5cf36d0b104eb1bec9df24,953ff434b4aa4c3aa094490e3b392b6b,178a736508c090a69181daded6120492,5088fe3df88f7b1af195bcef18d6067d,Total depth
AL-C3-Cistus,0.0,0.0,323.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13546
AL-C5-Cistus,31022.0,0.0,0.0,11927.0,9906.0,10002.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63599
AL-C6-Cistus,44.0,0.0,310.0,29.0,0.0,18.0,0.0,0.0,75.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9322
AL-E3-Cistus,0.0,0.0,934.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10109
AL-E5-Cistus,418.0,0.0,202.0,188.0,150.0,126.0,114.0,0.0,323.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24744
AL-E6-Cistus,276.0,0.0,1676.0,130.0,122.0,126.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12289
AS-C1-Cistus,914.0,0.0,6039.0,445.0,472.0,433.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30532
AS-C3-Cistus,561.0,0.0,5514.0,236.0,220.0,220.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30609
AS-C5-Cistus,3545.0,0.0,143.0,1595.0,1365.0,1325.0,16235.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50430
AS-E1-Cistus,696.0,0.0,2572.0,282.0,252.0,262.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22553


# Revision of outputs

In [ ]:
class Data:
    def __init__(self):
        self.qzv_in = "Assets/ancom_Species_clean_full_5.qzv"
        self.metadata = "Assets/metadata.tsv"
        self.metadata_column = "Species"        
        self.mode = "full"
        self.state = "clean"
        self.level = "5"
        self.relfreq_in = "Assets/absolute_numbers_lvl_5_clean_long.tsv"
args = Data()

In [ ]:
import argparse
import uuid
import os
import sys
import shutil
import pandas as pd
from qiime2 import Visualization
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)

def save_long_wide(df, filename, rowname, colname):
    """
    Generates tsv for a dataframe and for its transposed
    """
    df.to_csv(f"{filename}_row_{rowname}_col_{colname}.tsv", sep="\t")
    df.transpose().to_csv(f"{filename}_row_{colname}_col_{rowname}.tsv", sep="\t")
    return

def export_qzv(qzv_in, argument):
    # generate the tmp dir
    # export qzv to a tmp dir
    # get the needed dataframes
    # remove the tmp dir

    tmpdir = str(uuid.uuid4())

    while tmpdir in os.listdir():
        tmpdir = str(uuid.uuid4())

    qzv = Visualization.load(qzv_in)
    qzv.export_data(tmpdir)
    
    # ANCOM table
    df_ancom = pd.read_csv(f"{tmpdir}/ancom.tsv", sep="\t", index_col=0)
    
    # Data table
    df_data = pd.read_csv(f"{tmpdir}/data.tsv", sep="\t", index_col=0)
    newrow = pd.DataFrame.from_dict({argument : ["-","-"]}, orient="index", columns=["W","clr"])
    
    # Generate extra row to avoid NAs    
    df_data = pd.concat([newrow, df_data], axis=0)
    
    # remove the "w"
    df_ancom.drop(["W"], axis=1, inplace=True)
    
    # Generate extra row to avoid NAs
    newrow = pd.DataFrame.from_dict({argument : ["-"]}, orient="index", columns=["Reject null hypothesis"])
    df_ancom = pd.concat([newrow, df_ancom], axis=0)
    
    # Percent abundances
    df_percent_abundances = pd.read_csv(f"{tmpdir}/percent-abundances.tsv", sep = "\t", index_col=0)
    df_percent_abundances = df_percent_abundances.rename(index={"Group" : argument})
    
    shutil.rmtree(tmpdir)

    return df_ancom, df_data, df_percent_abundances

def get_significative_taxa(df):
    # Get differentially expressed taxa
    # Those with "Reject null hypothesis" set as True
    significative_taxa = df[df["Reject null hypothesis"] == True].index

    if len(significative_taxa) == 0:
        print(f"{args.mode}, {args.state}, lvl {args.level}, category {args.metadata_column}: no significative data found.")
        return None
    else:
        return list(significative_taxa)
    
def digest_name(string):
    name_as_list = string.replace("d__","domain: " ).replace("p__", "phylum: ").replace("c__", "class: " ).replace("o__", "order: " ).replace("f__","family: " ).replace("g__","genus: "  ).replace("s__","species: ").split(";")
    return name_as_list


In [ ]:
df_ancom, df_data, df_percent_abundances = export_qzv(args.qzv_in, args.metadata_column)

# generate first file
# Full ANCOM results: ancom data, percent_abundances
df_out_1 = pd.concat([df_ancom, df_data, df_percent_abundances], axis=1)

In [ ]:
df_out_1

In [ ]:
# get the significative data 
significative_taxa = get_significative_taxa(df_out_1)

# import relative abundances
rel_abs_df = pd.read_csv(args.relfreq_in, header=0, index_col=0, delimiter="\t")

# Import metadata
column_df = pd.DataFrame(pd.read_csv(args.metadata, header=0, index_col=0, delimiter="\t").pop(args.metadata_column)).transpose()

# Second file generated: ancom with the relative frequence


df_out_2 = pd.concat([df_ancom, df_data, pd.concat([pd.DataFrame(column_df), rel_abs_df], axis=0)], axis=1)

In [ ]:
# if there are any significative taxa
# generate the heatmap with dendrogram plot
if significative_taxa is not None:

    sig_tax_abundances = rel_abs_df.loc[significative_taxa, :]
    
    # change the headers of the table
    # get current names
    rownames = sig_tax_abundances.index
    
    # change current name into new future name
    # Get the values from the wanted columns
    newnames = [digest_name(item) for item in rownames]
    newnames = [f"{item[-1]}; {item[-2]}" if ("uncultured" in item[-1] and len(item) > 2) or ("__" in item[-1] and len(item) > 2) else item[-1] for item in newnames]
    
    namedict = { row : newname for row, newname in zip(rownames, newnames)}
    
    figure_df = sig_tax_abundances.rename(index=namedict)

    # associate color code to metadata
    color_codes = dict(zip(column_df.squeeze().unique(), ["#00AA5A", "#C0AB52", "#E16A86",  "#00A6CA",  "#C699E7", "#9A9A9A", "#65B891", "#F7934C", "#0B4F6C", "#F2E2D2", "#E1CE7A", "#646536", "#FFDD4A", "#EF3054", "#3D314A"]))
    col_colors = column_df.squeeze().map(color_codes)
    
    try:

        figure = plt.figure(figsize=(10,5))
        
        figure = sns.clustermap(figure_df,
                    col_colors=col_colors,
                    row_cluster=False,
                    dendrogram_ratio=(0, .15),
                    cbar_pos=(0.9, 0.1, .05, .25),
                    cmap="Greens",
                    figsize=(15,10),
                    )

        handles = [Patch(facecolor=color_codes[name]) for name in color_codes]
        plt.legend(handles, color_codes, title=args.metadata_column,
            bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure)

        figure.savefig(f"hmap_{args.metadata_column}_lvl_{args.level}_{args.state}_{args.mode}_xsamples_ytaxa.png")
        
        reverse_figure_df = figure_df.transpose()
        
        figure = sns.clustermap(reverse_figure_df,
                                row_colors=col_colors,
                                col_cluster=False,
                                dendrogram_ratio=(0.15, 0),
                                cbar_pos=(0.9, 0.1, .05, .20),
                                cmap="Greens",
                                figsize=(10,15),
                            )
        plt.legend(handles, color_codes, title=args.metadata_column,
            bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure)
        
        figure.savefig(f"hmap_{args.metadata_column}_lvl_{args.level}_{args.state}_{args.mode}__xtaxa_ysamples.png")

    except FloatingPointError:
    # This happens when data cannot be clustered
        print(f"{args.mode}, {args.state}, lvl {args.level}, category {args.metadata_column}: Could not cluster samples, generating unclustered heatmap instead.")

        figure = sns.clustermap(figure_df,
                    col_colors=col_colors,
                    row_cluster=False,
                    col_cluster=False,
                    cbar_pos=(0.9, 0.1, .05, .25),
                    cmap="Greens",
                    figsize=(15,10),
                    )     

        figure.savefig(f"hmap_unclustered_{args.metadata_column}_lvl_{args.level}_{args.state}_{args.mode}_xsamples_ytaxa.png")
        reverse_figure_df = figure_df.transpose()
        
        figure = sns.clustermap(reverse_figure_df,
                                row_colors=col_colors,
                                col_cluster=False,
                                row_cluster=False,
                                cbar_pos=(0.9, 0.1, .05, .20),
                                cmap="Greens",
                                figsize=(10,15),
                            )
                            
        figure.savefig(f"hmap_unclustered_{args.metadata_column}_lvl_{args.level}_{args.state}_{args.mode}__xtaxa_ysamples.png")



    # Third file
    # Only significative taxa involved
    relevant_rows = [args.metadata_column] + significative_taxa
    df_out_3 = df_out_2.loc[relevant_rows, :]

    # New df for Mode, level and state columns
    newcols = pd.DataFrame([
        ["-"] + [args.mode] * (df_out_3.shape[0]-1),
        ["-"] + [args.level] * (df_out_3.shape[0]-1),
        ["-"] + [args.state] * (df_out_3.shape[0]-1)
        ]).transpose()
    newcols.columns = ["Mode", "Level", "State"]
    newcols.index = df_out_3.index

    # Add the new columns
    df_out_3 = pd.concat([newcols, df_out_3], axis=1)
    save_long_wide(df_out_3, f"Significative_results_{args.metadata_column}_lvl_{args.level}_{args.state}_{args.mode}", "taxa", "ancom-samples")

else:
    sys.exit(0)

In [ ]:
df_out_3

# Uncollapsed ANCOM

In [ ]:
import argparse
import uuid
import os
import sys
import shutil
import pandas as pd
from qiime2 import Visualization
from qiime2 import Artifact
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)

def save_long_wide(df, filename, rowname, colname):
    """
    Generates tsv for a dataframe and for its transposed
    """
    df.to_csv(f"{filename}_row{rowname}_col{colname}.tsv", sep="\t")
    df.transpose().to_csv(f"{filename}_row{colname}_col{rowname}.tsv", sep="\t")
    return

def export_qzv(qzv_in, argument):
    # generate the tmp dir
    # export qzv to a tmp dir
    # get the needed dataframes
    # remove the tmp dir

    tmpdir = str(uuid.uuid4())

    while tmpdir in os.listdir():
        tmpdir = str(uuid.uuid4())

    qzv = Visualization.load(qzv_in)
    qzv.export_data(tmpdir)

    # ANCOM table
    df_ancom = pd.read_csv(f"{tmpdir}/ancom.tsv", sep="\t", index_col=0)
    
    # Data table
    df_data = pd.read_csv(f"{tmpdir}/data.tsv", sep="\t", index_col=0)
    newrow = pd.DataFrame.from_dict({argument : ["-","-"]}, orient="index", columns=["W","clr"])
    
    # Generate extra row to avoid NAs    
    df_data = pd.concat([newrow, df_data], axis=0)
    
    # remove the "w"
    df_ancom.drop(["W"], axis=1, inplace=True)
    
    # Generate extra row to avoid NAs
    newrow = pd.DataFrame.from_dict({argument : ["-"]}, orient="index", columns=["Reject null hypothesis"])
    df_ancom = pd.concat([newrow, df_ancom], axis=0)
    
    # Percent abundances
    df_percent_abundances = pd.read_csv(f"{tmpdir}/percent-abundances.tsv", sep = "\t", index_col=0)
    df_percent_abundances = df_percent_abundances.rename(index={"Group" : argument})

    shutil.rmtree(tmpdir)

    return df_ancom, df_data, df_percent_abundances

def get_significative_taxa(df):
    # Get differentially expressed taxa
    # Those with "Reject null hypothesis" set as True
    significative_taxa = df[df["Reject null hypothesis"] == True].index

    if len(significative_taxa) == 0:
        print("No significative data found.")
        return None
    else:
        return list(significative_taxa)

def digest_name(string):
    name_as_list = string.replace("d__","domain: " ).replace("p__", "phylum: ").replace("c__", "class: " ).replace("o__", "order: " ).replace("f__","family: " ).replace("g__","genus: "  ).replace("s__","species: ").split(";")
    return name_as_list

class Data:
    def __init__(self):
        self.ancom_qzv = "Assets/ancom_sample-origin_full.qzv"
        self.tax_qza = "Assets/taxonomy.qza"
        self.metadata_column = "sample-origin"
        self.metadata = "Assets/metadata.tsv"
        self.mode = "full"
        self.relfreq_in = "Assets/relative_numbers_full_long.tsv"

args = Data()

In [ ]:
# Import ANCOM result
df_ancom, df_data, df_percent_abundances = export_qzv(args.ancom_qzv, args.metadata_column)
df_out_1 = pd.concat([df_ancom, df_data, df_percent_abundances], axis=1)

# Import taxonomy
tax_df = Artifact.load(args.tax_qza).view(pd.DataFrame)

In [ ]:
# Generate new row so sample-origin goes first
# rowname = data
newrow = pd.DataFrame(["-"]*2).transpose()
newrow.columns = ["Consensus", "Taxon"]
newrow.index=[args.metadata_column]

# Add the new row
tax_df = tax_df.loc[list(df_out_1.index[1:])]
tax_df_meta = pd.concat([newrow, tax_df], axis=0)
tax_df_meta

In [ ]:
df_out_1 = pd.concat([tax_df_meta, df_out_1], axis=1)
df_out_1

In [ ]:
# Import relative abundances
rel_freq_df = pd.read_csv(args.relfreq_in, header=0, index_col=0, delimiter="\t")

# Import metadata
column_df = pd.read_csv(args.metadata, header=0, index_col=0, delimiter="\t").pop(args.metadata_column)

In [ ]:
df_out_2 = pd.concat([tax_df_meta, df_ancom, df_data, pd.concat([pd.DataFrame(column_df).transpose(), rel_freq_df], axis=0)], axis=1)
new_index = [args.metadata_column] + list(df_out_2["Taxon"])[1:]
ids = ["-"] + list(df_out_2.index)[1:]
df_out_2.index = new_index
df_out_2["Taxon"] = ids
df_out_2 = df_out_2.rename(columns={"Taxon" : "ID"})

# df_out_2.drop(["Taxon"], axis=1)

In [ ]:
significative_taxa = get_significative_taxa(df_out_2)

if significative_taxa is not None:
    
    rel_freq_df = pd.concat([tax_df, rel_freq_df], axis=1)
    rel_freq_df.index = list(rel_freq_df["Taxon"])
    rel_freq_df = rel_freq_df.drop(["Taxon","Consensus"], axis=1)
    
    sig_tax_abundances = rel_freq_df.loc[significative_taxa, :]
    
    # change the headers of the table
    # get current names
    rownames = sig_tax_abundances.index
    
    # change current name into new future name
    # Get the values from the wanted columns
    newnames = [digest_name(item) for item in rownames]
    newnames = [f"{item[-1]}; {item[-2]}" if ("uncultured" in item[-1] and len(item) > 2) or ("__" in item[-1] and len(item) > 2) else item[-1] for item in newnames]
    
    namedict = { row : newname for row, newname in zip(rownames, newnames)}
    
    figure_df = sig_tax_abundances.rename(index=namedict)
    
    # associate color code to metadata
    color_codes = dict(zip(column_df.squeeze().unique(), ["#00AA5A", "#C0AB52", "#e16a86",  "#00A6CA",  "#C699E7", "grey"]))
    col_colors = column_df.squeeze().map(color_codes)
    
    figure = sns.clustermap(figure_df,
                  col_colors=col_colors,
                  row_cluster=False,
                  dendrogram_ratio=(0, .15),
                  cbar_pos=(0.9, 0.1, .05, .25),
                  cmap="Greens",
                  figsize=(15,10),
                  )

    handles = [Patch(facecolor=color_codes[name]) for name in color_codes]
    plt.legend(handles, color_codes, title=args.metadata_column,
           bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure)

    figure.savefig(f"hmap_{args.metadata_column}_unleveled_raw_{args.mode}_xsamples_ytaxa.png")
    
    reverse_figure_df = figure_df.transpose()
    
    figure = sns.clustermap(reverse_figure_df,
                            row_colors=col_colors,
                            col_cluster=False,
                            dendrogram_ratio=(0.15, 0),
                            cbar_pos=(0.9, 0.1, .05, .20),
                            cmap="Greens",
                            figsize=(10,15),
                           )
    plt.legend(handles, color_codes, title=args.metadata_column,
        bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure)
    
    
    
    figure.savefig(f"hmap_{args.metadata_column}_uncollapsed_raw_{args.mode}_xtaxa_ysamples.png")

    # Third file
    # Only significative taxa involved
    relevant_rows = [args.metadata_column] + significative_taxa
    df_out_3 = df_out_2.loc[relevant_rows, :]

    # New df for Mode, level and state columns
    newcols = pd.DataFrame([
        ["-"] + [args.mode] * (df_out_3.shape[0]-1),
        ["-"] + ["uncollapsed"] * (df_out_3.shape[0]-1),
        ["-"] + ["raw"] * (df_out_3.shape[0]-1)
        ]).transpose()
    newcols.columns = ["Mode", "Level", "State"]
    newcols.index = df_out_3.index

    # Add the new columns
    df_out_3 = pd.concat([newcols, df_out_3], axis=1).drop(["Consensus", "ID"], axis=1)
    save_long_wide(df_out_3, f"significative_results_{args.metadata_column}_uncollapsed_raw_{args.mode}", "taxa", "ancom-samples")

In [ ]:
df_out_3

# Collapsed ancom

In [ ]:
import argparse
import uuid
import os
import shutil
import pandas as pd
from qiime2 import Visualization
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)

def save_long_wide(df, filename, rowname, colname):
    """
    Generates tsv for a dataframe and for its transposed
    """
    df.to_csv(f"{filename}_row{rowname}_col{colname}.tsv", sep="\t")
    df.transpose().to_csv(f"{filename}_row{colname}_col{rowname}.tsv", sep="\t")
    return

def export_qzv(qzv_in, argument):
    # generate the tmp dir
    # export qzv to a tmp dir
    # get the needed dataframes
    # remove the tmp dir

    tmpdir = str(uuid.uuid4())

    while tmpdir in os.listdir():
        tmpdir = str(uuid.uuid4())

    qzv = Visualization.load(qzv_in)
    qzv.export_data(tmpdir)
    
    # ANCOM table
    df_ancom = pd.read_csv(f"{tmpdir}/ancom.tsv", sep="\t", index_col=0)
    
    # Data table
    df_data = pd.read_csv(f"{tmpdir}/data.tsv", sep="\t", index_col=0)
    newrow = pd.DataFrame.from_dict({argument : ["-","-"]}, orient="index", columns=["W","clr"])
    
    # Generate extra row to avoid NAs    
    df_data = pd.concat([newrow, df_data], axis=0)
    
    # remove the "w"
    df_ancom.drop(["W"], axis=1, inplace=True)
    
    # Generate extra row to avoid NAs
    newrow = pd.DataFrame.from_dict({argument : ["-"]}, orient="index", columns=["Reject null hypothesis"])
    df_ancom = pd.concat([newrow, df_ancom], axis=0)
    
    # Percent abundances
    df_percent_abundances = pd.read_csv(f"{tmpdir}/percent-abundances.tsv", sep = "\t", index_col=0)
    df_percent_abundances = df_percent_abundances.rename(index={"Group" : argument})
    
    shutil.rmtree(tmpdir)

    return df_ancom, df_data, df_percent_abundances

def get_significative_taxa(df):
    # Get differentially expressed taxa
    # Those with "Reject null hypothesis" set as True
    significative_taxa = df[df["Reject null hypothesis"] == True].index

    if len(significative_taxa) == 0:
        print("No significative data found.")
        return None
    else:
        return list(significative_taxa)
    
def digest_name(string):
    name_as_list = string.replace("d__","domain: " ).replace("p__", "phylum: ").replace("c__", "class: " ).replace("o__", "order: " ).replace("f__","family: " ).replace("g__","genus: "  ).replace("s__","species: ").split(";")
    return name_as_list


In [ ]:
qzv_in = "Assets/ancom_sample-origin_clean_full_6.qzv"
metadata = "Assets/metadata.tsv"
metadata_column = "sample-origin"
rel_freq_in = "Assets/relative_numbers_lvl_6_clean_long.tsv"

# generate the path where the relative abundances will be
# rel_abundances_path = f"../../09-qiime2_collapse_numbers/{args.mode}/{args.state}/lvl_{args.level}/{args.state}/relative_numbers_lvl_{args.level}_{args.state}_long.tsv"

df_ancom, df_data, df_percent_abundances = export_qzv(qzv_in, metadata_column)

# generate first output
df_out_1 = pd.concat([df_ancom, df_data, df_percent_abundances], axis=1)
# save_long_wide(df_out_1, "_")

# get the significative data 
significative_taxa = get_significative_taxa(df_out_1)

# import relative abundances
rel_abs_df = pd.read_csv(rel_freq_in, header=0, index_col=0, delimiter="\t")

# Import metadata
column_df = pd.read_csv(metadata, header=0, index_col=0, delimiter="\t").pop(metadata_column)
df_out_2 = pd.concat([df_ancom, df_data, pd.concat([pd.DataFrame(column_df).transpose(), rel_abs_df], axis=0)], axis=1)


# if there are any significative taxa
# generate the heatmap with dendrogram plot
if significative_taxa is not None:
    
    sig_tax_abundances = rel_abs_df.loc[significative_taxa, :]
    
    # change the headers of the table
    # get current names
    rownames = sig_tax_abundances.index
    
    # change current name into new future name
    # Get the values from the wanted columns
    newnames = [digest_name(item) for item in rownames]
    newnames = [f"{item[-1]}; {item[-2]}" if "uncultured" in item[-1] and len(item) > 2 else item[-1] for item in newnames]
    
    namedict = { row : newname for row, newname in zip(rownames, newnames)}
    
    figure_df = sig_tax_abundances.rename(index=namedict)
    
    # associate color code to metadata
    color_codes = dict(zip(column_df.unique(), ["#00AA5A", "#C0AB52", "#e16a86",  "#00A6CA",  "#C699E7", "grey"]))
    col_colors = column_df.map(color_codes)
    
    figure = sns.clustermap(figure_df,
                  col_colors=col_colors,
                  row_cluster=False,
                  dendrogram_ratio=(0, .15),
                  cbar_pos=(0.9, 0.1, .05, .25),
                  cmap="Greens",
                  figsize=(15,10),
                  )
    handles = [Patch(facecolor=color_codes[name]) for name in color_codes]
    plt.legend(handles, color_codes, title=metadata_column,
           bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure)

    figure.savefig("hmap_xsamples_ytaxa.png")
    
    reverse_figure_df = figure_df.transpose()
    
    figure = sns.clustermap(reverse_figure_df,
                            row_colors=col_colors,
                            col_cluster=False,
                            dendrogram_ratio=(0.15, 0),
                            cbar_pos=(0.9, 0.1, .05, .20),
                            cmap="Greens",
                            figsize=(10,15),
                           )
    plt.legend(handles, color_codes, title=metadata_column,
       bbox_to_anchor=(1, 1), bbox_transform=plt.gcf().transFigure)

    
    figure.savefig("hmap_xtaxa_ysamples.png")
        
    


In [ ]:
column_df

In [ ]:
df_out_1

In [ ]:
df_out_2

In [ ]:
level = 7
state = "clean"
mode = "full" 

relevant_rows = [metadata_column] + significative_taxa

df_out_3 = df_out_2.loc[relevant_rows, :]

# New row for columns
level_cols = pd.DataFrame([
    ["-"] + [mode] * (df_out_3.shape[0]-1),
    ["-"] + [level] * (df_out_3.shape[0]-1),
    ["-"] + [state] * (df_out_3.shape[0]-1)    
]).transpose()

level_cols

level_cols.columns = ["Mode","Level","State"]
level_cols.index = df_out_3.index

df_out_3 = pd.concat([level_cols, df_out_3], axis=1)

In [ ]:
df_out_3

# Abs_count, Rel_count, Prevalence, Cleaning, Repeat

In [ ]:
from qiime2 import Artifact
import pandas as pd

import sys
import os

In [ ]:
def save_long_wide(df, filename):
    """
    Generates tsv for a dataframe and for its transposed
    """
    
    df.to_csv(f"{filename}_wide.tsv", sep="\t")
    df.transpose().to_csv(f"{filename}_long.tsv", sep="\t")
    return

def relative_abundances(df):
    """
    Obtain the relative abundance of the otus
    """
    
    df["Total"] = df.sum(axis=1)
    rownum, colnum = df.shape
    for row in range(rownum):
        for col in range(colnum-1):
            df.iloc[row, col] = df.iloc[row, col] * 100 / df.iloc[row, colnum-1]

    df.drop("Total", axis=1, inplace=True)   
    
    return df
    
def normalize_dataframe(dataframe, criteria=0):
    """
    Change the dataframe to an absence-presence matrix
    based on a criteria (by now, a number)
    """
    
    row_number, col_number = dataframe.shape
    
    for row in range(0, row_number):
        for col in range(0, col_number):
            if dataframe.iloc[row, col] >= criteria:
                
                print(f"{dataframe.iloc[row, col]} is considered 1")
                
                dataframe.iloc[row, col] = 1
            else:
                print(f"{dataframe.iloc[row, col]} is considered 0")

                dataframe.iloc[row, col] = 0
                
    return dataframe

def create_category_dict(metadata):
    """
    Create, from the metadata dataframe, a dict with
    key: category; val: values in that category
    if only one category, it wont be taken into account
    """
    valid_categories = dict()
    category_names_list = list(metadata.columns)

    # get all different possibilities for each metadata column
    for col_index in range(metadata.shape[1]):
        
        # list from a set to avoid repeating
        groups = (list(set(metadata[category_names_list[col_index]])))
        
        # if more than 1 different category, add it to the dict
        if len(groups) > 1:
            category_name = category_names_list[col_index]
            valid_categories[category_name] = [item for item in groups]

    return valid_categories, category_names_list

def prevalences(df, metadata):
    """
    Calculate the prevalence for each group
    """
    
    df_prev = pd.concat([df, metadata], axis=1)
    category_dict, category_names_list = create_category_dict(metadata)
    
    for category, values in category_dict.items():
    
        prevalence_per_value = []

        for value in values:

            # Drop metadata columns
            sub_df = df_prev[df_prev[category] == value].drop(category_names_list, axis=1)
            
            # Normalize (0: absence, 1: presence)
            norm_df = normalize_dataframe(sub_df, criteria=1)
            norm_df.loc["Prevalence"] = norm_df.sum(axis=0)
            
            
            norm_df.transpose().to_csv("normalizada.tsv",sep="\t")


            row_number, col_number = norm_df.shape
        
            for column in range(0, col_number):
                # Get the relative abundance of each taxon on each group
                norm_df.iloc[row_number-1, column] = norm_df.iloc[row_number-1, column]*100/(row_number-1)
                norm_df.rename({"Prevalence":value}, axis=0, inplace=True)

            prevalence_per_value.append(norm_df.loc[value].to_frame().transpose())        
        
        prevalence_df = pd.concat(prevalence_per_value)
        
        save_long_wide(prevalence_df, f"prevalence")  
    

def clean_dataframe(df):
    """
    Remove the columns ending with ;__
    """
    
    df = df.loc[:,~df.columns.str.endswith(";__")]

    return df


def artifact_from_df(df_in, filename):
    
    clean_qza = Artifact.import_data("FeatureTable[Frequency]", df_in)
    clean_qza.save(f"{filename}.qza")
    
    return

In [ ]:
qza_in = "Assets/collapsed_raw_full_table_lvl_5.qza"

In [ ]:
df = Artifact.load(qza_in).view(pd.DataFrame)

In [ ]:
save_long_wide(df,"absolute_numbers")

In [ ]:
rel_df = relative_abundances(df)
save_long_wide(rel_df,"relative_numbers")

In [ ]:
metadata_file = "Assets/metadata.tsv"

metadata = pd.read_csv(
    metadata_file,
    sep='\t',
    header=0,
    index_col=0
    )
prevalences(df, metadata)

In [ ]:
clean = clean_dataframe(df)

artifact_from_df(clean, "table_clean")

# Prevalence

In [ ]:
import sys
import os

import pandas as pd
from qiime2 import Artifact

In [ ]:
def normalize_dataframe(dataframe, criteria=0):
    """
    Change the dataframe to an absence-presence matrix
    based on a criteria (by now, a number)
    """
    
    row_number, col_number = dataframe.shape
    
    for row in range(0, row_number):
        for col in range(0, col_number):
            if dataframe.iloc[row, col] >= criteria:
                dataframe.iloc[row, col] = 1
            else:
                dataframe.iloc[row, col] = 0
                
    return dataframe

def create_category_dict(metadata):
    """
    Create, from the metadata dataframe, a dict with
    key: category; val: values in that category
    if only one category, it wont be taken into account
    """
    valid_categories = dict()
    category_names_list = list(metadata.columns)

    # get all different possibilities for each metadata column
    for col_index in range(metadata.shape[1]):
        
        # list from a set to avoid repeating
        groups = (list(set(metadata[category_names_list[col_index]])))
        
        # if more than 1 different category, add it to the dict
        if len(groups) > 1:
            category_name = category_names_list[col_index]
            valid_categories[category_name] = [item for item in groups]

    return valid_categories, category_names_list

In [ ]:
qza_in = "Assets/collapsed_raw_full_table_lvl_5.qza"
metadata_file = "Assets/metadata.tsv"
lvl = 6

In [ ]:
try:
    os.mkdir(f"prevalence")
except:
    pass
qza = Artifact.load(qza_in)
counts = qza.view(pd.DataFrame)

metadata = pd.read_csv(
    metadata_file,
    sep='\t',
    header=0,
    index_col=0
    )
full_df = pd.concat([metadata, counts], axis=1)

In [ ]:
counts

In [ ]:
metadata

In [ ]:
full_df

In [ ]:
valid_categories, category_names_list = create_category_dict(metadata)

In [ ]:
valid_categories

In [ ]:
category_names_list

In [ ]:
for category, values in valid_categories.items():
    
    prevalence_per_value = []
    
    # print(category)
    for value in values:
        # print(value)
        # Drop metadata columns
        sub_df = full_df[full_df[category] == value].drop(category_names_list, axis=1)
        # Normalize (0: absence, 1: presence)
        norm_df = normalize_dataframe(sub_df, criteria=1)
        norm_df.loc["Prevalence"] = norm_df.sum(axis=0)
        
        row_number, col_number = norm_df.shape
        
        # data
        
        for column in range(0, col_number):
            # Get the relative abundance of each taxon on each group
            norm_df.iloc[row_number-1, column] = norm_df.iloc[row_number-1, column]*100/(row_number-1)
        norm_df.rename({"Prevalence":value}, axis=0, inplace=True)
        
        prevalence_per_value.append(norm_df.loc[value].to_frame().transpose())        
        
    prevalence_df = pd.concat(prevalence_per_value)
    
    prevalence_df.to_csv(f"prevalence_lvl_{lvl}_{category}_{value}_wide.tsv", sep="\t")
    prevalence_df.transpose().to_csv(f"prevalence_lvl_{lvl}_{category}_{value}_long.tsv", sep="\t")
    
    
        

In [ ]:
norm_df

In [ ]:
prevalence_df

# RELATIVE COUNTS

In [ ]:
import shutil
import os
import sys

import pandas as pd
from qiime2 import Artifact

In [ ]:
filename = "Assets/collapsed_raw_full_table_lvl_5.qza"
outdir = "lvl6"

In [ ]:
# Open visualization
qza_artifact = Artifact.load(filename)
df = qza_artifact.view(pd.DataFrame)

In [ ]:
df

In [ ]:
df["Total"] = df.sum(axis=1)

In [ ]:
df

In [ ]:
rownum, colnum = df.shape
for row in range(rownum):
    for col in range(colnum-1):
        df.iloc[row, col] = df.iloc[row, col] * 100 / df.iloc[row, colnum-1]

df.drop("Total", axis=1)
df.to_csv("relativ_freq.tsv", sep="\t")

In [ ]:
df

In [ ]:
df["Total"] = df.sum()
# Delete unwanted dirs & files
# Hardcoded but its always the same so
dirs_to_del = ["css", "js", "q2templateassets"]

for folder in dirs_to_del:
    shutil.rmtree(f"{outdir}/{folder}")

files_to_del = ["index.html"]
for file in files_to_del:
    os.remove(f"{outdir}/{file}")

In [ ]:
df = pd.read_csv(f"{outdir}/metadata.tsv", sep="\t", header=0, index_col=0)

df = df.drop("#q2:types")
df.to_csv(f"{table_name}.tsv", sep="\t")
df.transpose().to_csv(f"{table_name}_long.tsv", sep="\t")

# ANCOM

In [ ]:
import sys
import os
import shutil

import pandas as pd
from qiime2 import Visualization
import seaborn as sns; sns.set_theme(color_codes=True)

In [ ]:
# ANCOM qzv file
qzv_in = "Assets/ancom_sample-origin_full.qzv"
tmpdir = "tmp"
rel_feat_table = "Assets/relativ_freq.tsv"

In [ ]:
 def export_qzv(qzv_in, tmpdir):
    # export qzv to a tmp dir
    # get the needed dataframes
    # remove the tmp dir
    qzv = Visualization.load(qzv_in)
    qzv.export_data(tmpdir)
    
    # ancom table
    df_ancom = pd.read_csv(f"{tmpdir}/ancom.tsv", sep="\t", index_col=0)
    
    # Data table
    df_data = pd.read_csv(f"{tmpdir}/data.tsv", sep="\t", index_col=0)
    
    # Add extra row to avoid NAs
    df_data.loc["Group"] = 2 * ["-"]
    
    # remove the "w"
    df_ancom.drop(["W"], axis=1, inplace=True)

    # Percent abundances
    df_percent_abundances = pd.read_csv(f"{tmpdir}/percent-abundances.tsv", sep = "\t", index_col=0)

    shutil.rmtree(tmpdir)

    return df_ancom, df_data, df_percent_abundances

In [ ]:
def get_significative_taxa(df):
    # Get differentially expressed taxa
    significative_df = df[df["Reject null hypothesis"] == True].loc[:,["Reject null hypothesis", "clr", "W"]]
    significative_taxa = list(significative_df.index)

    if len(significative_taxa) == 0:
        print("No significative data found.")
        return None
    else:
        return significative_taxa

In [ ]:
relfq_df = pd.read_csv(rel_feat_table, sep="\t", header=0, index_col=0)
relfq_df

In [ ]:
relqf_df = relfq_df.transpose().loc[significative_taxa]

In [ ]:
relqf_df.loc["Metadata group"] = ["a"] * 26
relqf_df

In [ ]:
final_df = pd.concat([significative_df,relqf_df], axis=1)

In [ ]:
final_df